# 0. 패키지 불러오기

In [1]:
import numpy as np
import scipy
import implicit
import os
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

# 1. 데이터 준비 및 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendation_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendation_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies['title'] = movies['title'].str.lower()
movies['genre'] = movies['genre'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy


검색을 쉽게하기위해 영화제목과 장르를 전부 소문자로 바꿔줍니다.

In [8]:
print('ratings 결측치 수:',ratings.isnull().sum())
print('movies 결측치 수:', movies.isnull().sum())

ratings 결측치 수: user_id      0
movie_id     0
counts       0
timestamp    0
dtype: int64
movies 결측치 수: movie_id    0
title       0
genre       0
dtype: int64


In [9]:
# 장르는 중복일 수 있어도 영화 이름이 중복이면 안됩니다. 
print('movies의 중복치 수 :',movies['title'].duplicated().sum())

movies의 중복치 수 : 0


결측치나 중복치가 있는지 확인해줍니다.

# 2. EDA

In [10]:
print('유저수 :', ratings['user_id'].nunique())
print('영화수 :', movies['title'].nunique())

유저수 : 6039
영화수 : 3883


In [11]:
# 영화id별 유저id수를 movie_count변수에 넣습니다.
movie_count = ratings.groupby('movie_id')['user_id'].count()

# 내림차순으로 30개를 나열하고 해당 영와id를 넘파이 배열형태로 mv_id_30변수에 저장합니다.
mv_id_30 = movie_count.sort_values(ascending=False)[:30].index.values

# loc를 이용해서 id에 해당하는 영화 이름을 출력합니다.
movies.loc[mv_id_30,'title']

2858                               brief encounter (1946)
260                              ladybird ladybird (1994)
1196                                         alien (1979)
1210                                   raging bull (1980)
2028               something wicked this way comes (1983)
589                      silence of the lambs, the (1991)
593                                   pretty woman (1990)
1198                 big blue, the (le grand bleu) (1988)
1270                        some kind of wonderful (1987)
2571                                      superman (1978)
480                                         lassie (1994)
2762                            dog of flanders, a (1999)
608                                pallbearer, the (1996)
110                            rumble in the bronx (1995)
1580                                    wishmaster (1997)
527                             secret garden, the (1993)
1197                              army of darkness (1993)
2396          

인기순으로 영화목록 30개를 출력해보겠습니다.

In [12]:
# alien, toy story, superman, heathers, priest
my_favorite = [1196, 1, 2571, 1265, 296]

# 제 자신의 id를 9999로 설정하고, 각 영화에 대한 별점은 5점을 주겠습니다.
my_list = pd.DataFrame({'user_id': [9999]*5, 'movie_id': my_favorite, 'counts': [5]*5})
ratings = ratings.append(my_list)
ratings.tail(10)

,user_id,movie_id,counts,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,9999,1196,5,NaN
1,9999,1,5,NaN
2,9999,2571,5,NaN
3,9999,1265,5,NaN
4,9999,296,5,NaN


우리는 본인의 영화취향과 가장 유사한 또다른 영화를 추천받고 싶습니다. 넷플릭스 등 추천 시스템들은 이를 위해서 처음 가입하는 사용자의 취향인 영화정보를 5개 이상 입력받는 과정을 거치게 하는 경우가 많습니다.

이와 동일한 과정을 위해 위 데이터셋에 제가 좋아하는 영화5개를 추가하겠습니다.

## 모델에 활용하기 위한 전처리
사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 movie 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 indexing이라고 합니다.

In [13]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie_id 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movid_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6039,117,5,NaN
1,6039,40,5,NaN
2,6039,124,5,NaN
3,6039,110,5,NaN


# 3. CSR matrix
CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다.

CSR matrix를 직접 만들어 봅시다.

In [15]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_ratings = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movie))
csr_ratings

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 4. Model Training
MF모델을 implicit패키지를 사용하여 학습해봅시다.

implicit패키지는 암묵적 데이터셋을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.

이 패키지에 구현된 als(AlternatingLeastSquares)모델을 사용하겠습니다. MF에서 쪼개진 두 feature matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른쪽을 학습하는 방식을 번갈아 수행하는 ALS방식이 효과적인 것으로 알려져있습니다.

In [16]:
# implicit 라이브러리에서 권장하고 있는 부분입니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [17]:
als_model = AlternatingLeastSquares(factors=300, regularization=0.01, use_gpu=False, iterations=40, dtype=np.float32)

ALS클래스의 '\__init__\'파라미터를 살펴보겠습니다.

- factors : 유저와 아이템의 벡터를 몇차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU를 사용할 것인지
- iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [18]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [19]:
als_model.fit(csr_ratings_transpose)

  0%|          | 0/40 [00:00<?, ?it/s]

학습이 끝났습니다. 이제 2가지를 살펴보도록하겠습니다.

- me벡터와 alien의 벡터를 어떻게 만들고 있는지
- 두 벡터를 곱하면 어떤 값이 나오는지

In [20]:
me, alien = user_to_idx[9999], movie_to_idx[1196]
me_vector, alien_vector = als_model.user_factors[me], als_model.item_factors[alien]

In [21]:
me_vector

array([-2.01052010e-01, -1.68140918e-01,  2.77602255e-01, -3.11742350e-02,
       -8.53191540e-02,  4.89291400e-01, -1.06989413e-01,  1.22794002e-01,
       -1.81463912e-01, -1.96494870e-02, -4.96315770e-02, -2.74184376e-01,
       -2.98992664e-01, -1.12727560e-01, -7.92355388e-02, -6.09653354e-01,
       -1.26788825e-01, -2.20811382e-01, -8.21911454e-01, -2.08423346e-01,
        1.09285556e-01, -4.89238590e-01,  2.50157386e-01,  3.47295493e-01,
        8.55798721e-02, -2.88565934e-01, -1.65121242e-01,  6.22698106e-02,
       -4.29313183e-02, -9.43250358e-02,  1.48589939e-01,  2.82914668e-01,
       -3.44023585e-01, -1.78415582e-01,  1.78776070e-01, -2.53866669e-02,
       -2.55468130e-01,  4.10348862e-01, -1.34143576e-01, -4.17856090e-02,
       -1.12958208e-01, -1.75790712e-01, -2.89460093e-01, -3.59140724e-01,
       -8.79026502e-02,  6.68172538e-01, -3.21096450e-01, -3.93976957e-01,
       -2.25536060e-02,  8.15544501e-02, -2.58071035e-01, -2.50044852e-01,
        6.45533979e-01,  

In [22]:
alien_vector

array([-0.00246876,  0.02221882,  0.03505241,  0.00843004,  0.0010936 ,
       -0.00974372, -0.00691387,  0.0240144 ,  0.00791384,  0.01918036,
        0.00039211, -0.02633163, -0.01950226, -0.00174731,  0.01666697,
       -0.00711615,  0.00278425, -0.01252815, -0.02421623, -0.02656219,
        0.00510596, -0.02309096,  0.02350282, -0.00475428, -0.00907645,
       -0.0076217 , -0.00072826,  0.01301471,  0.0211603 , -0.03443914,
       -0.00944594, -0.00663843, -0.01614538,  0.00476445,  0.00981977,
        0.01015685, -0.02177292,  0.02645059, -0.00124668,  0.02372753,
        0.02643981, -0.01019626,  0.01668161, -0.01953787, -0.00364409,
        0.01559885,  0.0022545 ,  0.00170806,  0.01709651, -0.02021079,
        0.02507648, -0.0300436 ,  0.04353349,  0.03995365,  0.0442072 ,
        0.01780226,  0.01243417,  0.00618821, -0.02520622,  0.00576079,
       -0.00980566,  0.01463096,  0.011481  ,  0.014301  ,  0.01504409,
        0.03136016, -0.02414943,  0.02705514,  0.02504139,  0.01

In [23]:
# me와 alien을 내적하는 코드
np.dot(me_vector, alien_vector)

0.8012814

선호도가 꽤 높게 나왔습니다. 잘 훈련된 것 같습니다.

이렇게 학습된 모델을가지고 toy story에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다.

참고로 toy story는 이전에 제가 직접 별점5개를 준 영화입니다.

In [24]:
# movie_to_idx[1]은 toy story를 뜻합니다.
toy_story = movie_to_idx[1]
toy_vector = als_model.item_factors[toy_story]
np.dot(me_vector, toy_vector)

0.84533244

역시나 선호도가 높게나왔네요.

# 5. 내가 좋아하는 영화와 비슷한 영화를 추천받기
ALS클래스에 구현되어 있는 similar_items메서드를 통하여 비슷한 영화를 찾습니다. 처음으로는 제가 좋아하는 alien으로 찾아보겠습니다.

In [25]:
favorite_movie_idx = 1196 # alien 인덱스
movie_id = movie_to_idx[favorite_movie_idx]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(117, 1.0),
 (44, 0.46208465),
 (64, 0.36175212),
 (651, 0.30493712),
 (1993, 0.2635562),
 (3487, 0.26317015),
 (2763, 0.25828785),
 (2038, 0.25726062),
 (3458, 0.25486493),
 (120, 0.25449628),
 (3619, 0.25411862),
 (3493, 0.25293067),
 (3600, 0.25205463),
 (2779, 0.25106364),
 (3499, 0.25103754)]

(movie의 id, 유사도)Tuple을 반환하고있습니다. 하지만 가장 유사한 영화의 유사도가 0.4정도밖에 안되네요. 그래도 일단 movie의 id를 다시 movie의 title로 매핑 시켜주겠습니다.

In [26]:
# movie_to_idx를 뒤집어, index로부터 movie이름을 얻는 dict를 생성합니다.
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
mv_id = [idx_to_movie[i[0]] for i in similar_movie]
mv_id

[1196,
 260,
 1210,
 1200,
 885,
 3374,
 1471,
 84,
 2127,
 1198,
 3530,
 3595,
 3748,
 1134,
 2887]

In [27]:
movies['title'][mv_id]

1196                                         alien (1979)
260                              ladybird ladybird (1994)
1210                                   raging bull (1980)
1200            killer, the (die xue shuang xiong) (1989)
885                        for whom the bell tolls (1943)
3374                                 born american (1986)
1471                       8 heads in a duffel bag (1997)
84                              angels and insects (1995)
2127                                     knock off (1998)
1198                 big blue, the (le grand bleu) (1988)
3530                                anchors aweigh (1945)
3595            puppet master 5: the final chapter (1994)
3748                                   pot o' gold (1941)
1134    return of martin guerre, the (retour de martin...
2887                      someone to watch over me (1987)
Name: title, dtype: object

# 6. 내가 가장 좋아할 만한 영화들을 추천받기
ALS클래스에 구현되어 있는 recommend메서드를 통하여 제가 좋아할 만한 영화를 추천받습니다.\ filter_already_liked_items는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [28]:
user = user_to_idx[9999]

#recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_ratings, N=20, filter_already_liked_items=True)
movie_recommended

[(44, 0.36736852),
 (64, 0.32716677),
 (50, 0.23906273),
 (92, 0.21503773),
 (269, 0.18486807),
 (48, 0.18479452),
 (233, 0.16532359),
 (289, 0.16502778),
 (651, 0.1627573),
 (157, 0.1606977),
 (22, 0.14397359),
 (613, 0.14009213),
 (120, 0.13541587),
 (154, 0.13197236),
 (369, 0.13080077),
 (5, 0.13028231),
 (51, 0.13000374),
 (255, 0.1293566),
 (1088, 0.12745462),
 (33, 0.12722047)]

해당 인덱스들을 영화id로 바꿔봅시다.

In [29]:
[idx_to_movie[i[0]] for i in movie_recommended]

[260,
 1210,
 3114,
 589,
 1213,
 2028,
 50,
 1089,
 1200,
 318,
 1270,
 357,
 1198,
 1784,
 1674,
 1197,
 608,
 1923,
 708,
 588]

ladybird를 추천해주고있습니다. 모델은 왜 ladybird를 추천해줬을까요? als클래스에 구현된 explain메서드를 사용해 제가 기록을 남긴 데이터중 이 추천에 기여한 정도를 확인할 수 있습니다.

In [30]:
ladybird = movie_to_idx[1]
explain = als_model.explain(user, csr_ratings, itemid=ladybird)

이 method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환합니다. 어떤 영화들이 이 추천에 얼마나 기여하고 있는걸까요?

In [31]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(1, 0.7885233911207443),
 (2571, 0.028235225189519826),
 (1265, 0.024135607584047772),
 (296, 0.0011379710420549203),
 (1196, -0.0062046114495955965)]

토이스토리의 기여가 0.78로 대부분입니다.(index:1) 그런데 결과가 조금 이상합니다. filter_already_liked_items=True로 했는데 이미 평가한 아이템이 나왔습니다. 이것은 implicit 버전 0.4.2에서 생긴 버그라고합니다.

# 출처
https://github.com/seyoung0218/Exploration-AIFFEL/blob/main/Exploration13/Exploration13.ipynb



# 회고
- recommand system이 쉽지 않음을 깨달았다.
- csr matrix를 이해하기 쉽지 않았다.
- csr matrix를 방학 때 csr matrix를 이해해봐야겠다.